# UNDERWRITING DASHBOARD PROJECT

## A). PACKAGE IMPORT AND ENVIRONMENT SETUP

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
from datetime import datetime
import importlib
import subprocess
import time

# Setup project paths
PROJECT_ROOT = Path("C:/Users/MattBorgeson/OneDrive - B&R Capital/Programming Projects/Underwriting Dashboard")
sys.path.append(str(PROJECT_ROOT))

print(f"Project root directory: {PROJECT_ROOT}")
print(f"Current working directory: {os.getcwd()}")

# Utility function to reload modules
def reload_project_modules():
    """Reload project modules to ensure latest code changes are used."""
    modules_to_reload = [
        "src.data_processing.file_finder",
        "src.data_processing.excel_reader",
        "src.database.db_manager",
        "src.file_monitoring.monitor",
        "src.dashboard.app",
        "src.dashboard.components.filters",
        "src.dashboard.components.tables",
        "src.dashboard.components.maps",
        "src.dashboard.components.analytics",
        "src.dashboard.utils.responsive"
    ]
    
    for module_name in modules_to_reload:
        try:
            if module_name in sys.modules:
                importlib.reload(sys.modules[module_name])
                print(f"✓8 Reloaded module: {module_name}")
        except Exception as e:
            print(f"✗ Failed to reload module {module_name}: {str(e)}")

# Run this to reload all project modules if you've made changes
reload_project_modules()

## B). COMPONENT TESTING MODULES

### B.1). COMPONENT MODULE TEST -- FILE FINDER MODULE

In [ ]:
# Test File Finder Module
from src.data_processing.file_finder import find_underwriting_files

print("Running file finder to locate Excel underwriting models...")

# Find Excel files
include_files, exclude_files = find_underwriting_files()

print(f"Found {len(include_files)} files to include in analysis")
print(f"Found {len(exclude_files)} files to exclude from analysis")

# Display the first few included files
if include_files:
    print("\nFirst 5 included files:")
    for i, file_info in enumerate(include_files[:5]):
        print(f"{i+1}. {file_info['File Name']} - {file_info['Deal Stage Subdirectory Name']}")

### B.2). COMPONENT TEST -- EXCEL READER WITH ONE FILE

In [ ]:
# Test Excel Reader Module
from src.data_processing.excel_reader import process_excel_files

print("Testing Excel reader with the first file...")

# Process just the first file for testing
if include_files:
    test_file = [include_files[0]]
    print(f"Processing file: {test_file[0]['File Name']}")
    
    result_df = process_excel_files(test_file)
    
    if not result_df.empty:
        print(f"Successfully extracted {len(result_df.columns)} columns of data")
        
        # Display the first few columns
        print("\nFirst 10 columns:")
        print(list(result_df.columns)[:10])
        
        # Display a preview of the data
        print("\nData preview:")
        display(result_df.head(1))
    else:
        print("No data extracted from file")
else:
    print("No files found to process")

### B.3). COMPONENT TEST -- DATABASE OPERATIONS

In [ ]:
# Test Database Manager
from src.database.db_manager import setup_database, store_data, get_all_data

print("Testing database operations...")

# Setup the database
print("Setting up database...")
setup_database()

# Store data from the first file
if include_files and not result_df.empty:
    print(f"Storing data from {include_files[0]['File Name']} in database...")
    store_data(result_df)
    
    # Retrieve all data from database
    print("Retrieving all data from database...")
    all_data = get_all_data()
    
    print(f"Retrieved {len(all_data)} rows and {len(all_data.columns)} columns from database")
    
    # Display a few column names
    print("\nSample columns from database:")
    sample_cols = list(all_data.columns)[:10]
    for col in sample_cols:
        print(f"- {col}")
else:
    print("No data to store in database")

### B.4). COMPONENT TEST -- FILE MONITORING UTILITIES

In [ ]:
# Test File Monitoring (non-blocking)
from src.file_monitoring.monitor import find_uw_model_folder

print("Testing file monitoring utilities...")

# Test finding UW Model folders
if include_files:
    deal_folder = Path(include_files[0]["Absolute File Path"]).parent.parent
    print(f"Looking for UW Model folder in: {deal_folder}")
    
    uw_folder = find_uw_model_folder(deal_folder)
    
    if uw_folder:
        print(f"Found UW Model folder: {uw_folder}")
        
        # List files in the UW Model folder
        print("\nFiles in UW Model folder:")
        for i, file in enumerate(uw_folder.glob("*.*")):
            print(f"{i+1}. {file.name} ({file.stat().st_size:,} bytes)")
    else:
        print("UW Model folder not found")

### B.5). COMPONENT TEST -- LAUNCHING DASHBOARD

In [ ]:
# Run Dashboard in a separate process
def launch_dashboard():
    """Launch the Streamlit dashboard in a separate process."""
    dashboard_path = os.path.join(PROJECT_ROOT, "src", "dashboard", "app.py")
    
    print(f"Launching dashboard: {dashboard_path}")
    print("Dashboard will be available at: http://localhost:8501")
    print("Run this in a command prompt to start the dashboard:")
    print(f"streamlit run {dashboard_path}")
    
    # Uncomment to actually launch the dashboard (will block the notebook)
    subprocess.Popen(["streamlit", "run", dashboard_path])
    
    print("\nNote: For testing the dashboard, it's usually better to run it from a command prompt")
    print("This allows you to continue working in the notebook while the dashboard is running")

# Only run this cell if you want to launch the dashboard
launch_dashboard()

### B.6). COMPONENT TEST -- DATABASE SCHEMA AND CONTENTS

In [ ]:
# Explore Database Schema and Content
print("Exploring database schema and content...")

try:
    # Connect to the database
    db_path = os.path.join(PROJECT_ROOT, "database", "underwriting_models.db")
    conn = sqlite3.connect(db_path)
    
    # Get table names
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print(f"Database tables: {[table[0] for table in tables]}")
    
    # Get column info for the main table
    if tables:
        main_table = tables[0][0]
        cursor.execute(f"PRAGMA table_info({main_table})")
        columns = cursor.fetchall()
        
        print(f"\nColumns in {main_table} table:")
        for i, col in enumerate(columns[:20]):  # Show first 20 columns
            print(f"{i+1}. {col[1]} ({col[2]})")
        
        if len(columns) > 20:
            print(f"...and {len(columns) - 20} more columns")
        
        # Count rows
        cursor.execute(f"SELECT COUNT(*) FROM {main_table}")
        row_count = cursor.fetchone()[0]
        print(f"\nTotal rows in {main_table}: {row_count}")
    
    conn.close()
        
except Exception as e:
    print(f"Error exploring database: {str(e)}")

### B.7). COMPONENT TEST -- ANALYTICS COMPONENTS

In [ ]:
# Test Analytics Components
# This will test the analytics components without running the full dashboard

# Import the analytics components
import importlib.util
import sys

print("Testing analytics components...")

try:
    # Check if the analytics file exists and load it
    analytics_path = os.path.join(PROJECT_ROOT, "src", "dashboard", "components", "analytics.py")
    
    if os.path.exists(analytics_path):
        print(f"Analytics file found: {analytics_path}")
        
        # Get data for testing
        test_data = get_all_data() if 'get_all_data' in locals() else None
        
        if test_data is not None and not test_data.empty:
            print(f"Data available for analytics: {len(test_data)} rows")
            
            # Display data columns available for analytics
            print("\nColumns available for analytics:")
            for col_type, keywords in {
                "Geographic": ["city", "state", "lat", "lon", "location"],
                "Performance": ["cap", "rate", "irr", "price", "value"],
                "Property": ["unit", "property", "building", "deal"],
                "Dates": ["date", "time"]
            }.items():
                matching_cols = [col for col in test_data.columns if any(kw in str(col).lower() for kw in keywords)]
                if matching_cols:
                    print(f"\n{col_type} columns:")
                    for col in matching_cols[:5]:  # Show first 5 of each type
                        print(f"- {col}")
                    if len(matching_cols) > 5:
                        print(f"...and {len(matching_cols) - 5} more")
            
            print("\nNote: To fully test the analytics components, run the dashboard with:")
            print(f"streamlit run {os.path.join(PROJECT_ROOT, 'src', 'dashboard', 'app.py')}")
        else:
            print("No data available for testing analytics components")
            print("Run the Excel reader and database steps first to load data")
    else:
        print(f"Analytics file not found at: {analytics_path}")
        print("Create the analytics.py file with the geographic analysis functions first")
        
except Exception as e:
    print(f"Error testing analytics components: {str(e)}")
    import traceback
    traceback.print_exc()